In [2]:
airlinesPath="file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/airlines.csv"
airportsPath="file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/airports.csv"
flightPath="file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/flights.csv"

In [4]:
airlines = sc.textFile(airlinesPath)

In [5]:
print(airlines)

file:////Users/madsh01/Personal/Workspace/spark/practice/datasets/FlightsData/airlines.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [6]:
airlines.collect()

['Code,Description',
 '"19031","Mackey International Inc.: MAC"',
 '"19032","Munz Northern Airlines Inc.: XY"',
 '"19033","Cochise Airlines Inc.: COC"',
 '"19034","Golden Gate Airlines Inc.: GSA"',
 '"19035","Aeromech Inc.: RZZ"',
 '"19036","Golden West Airlines Co.: GLW"',
 '"19037","Puerto Rico Intl Airlines: PRN"',
 '"19038","Air America Inc.: STZ"',
 '"19039","Swift Aire Lines Inc.: SWT"',
 '"19040","American Central Airlines: TSF"',
 '"19041","Valdez Airlines: VEZ"',
 '"19042","Southeast Alaska Airlines: WEB"',
 '"19043","Altair Airlines Inc.: AAR"',
 '"19044","Chitina Air Service: CHI"',
 '"19045","Marco Island Airways Inc.: MRC"',
 '"19046","Caribbean Air Services Inc.: OHZ"',
 '"19047","Sundance Airlines: PRO"',
 '"19048","Seair Alaska Airlines Inc.: SAI"',
 '"19049","Southeast Airlines Inc.: SLZ"',
 '"19050","Alaska Aeronautical Indust.: AAZ"',
 '"19051","Imperial Airlines Inc.: IMP"',
 '"19052","Trans Western Airlines Utah: TWA"',
 '"19053","Wright Airlines Inc.: WRT"',
 '"19

In [7]:
airlines.first()

'Code,Description'

In [8]:
airlines.take(10)

['Code,Description',
 '"19031","Mackey International Inc.: MAC"',
 '"19032","Munz Northern Airlines Inc.: XY"',
 '"19033","Cochise Airlines Inc.: COC"',
 '"19034","Golden Gate Airlines Inc.: GSA"',
 '"19035","Aeromech Inc.: RZZ"',
 '"19036","Golden West Airlines Co.: GLW"',
 '"19037","Puerto Rico Intl Airlines: PRN"',
 '"19038","Air America Inc.: STZ"',
 '"19039","Swift Aire Lines Inc.: SWT"']

In [9]:
airlines.count()

1580

In [23]:
airlinesWoHdr= airlines.filter(lambda line : "Description" not in line)

In [12]:
airlinesWoHdr.take(1)

['"19031","Mackey International Inc.: MAC"']

In [13]:
airlinesParsed= airlinesWoHdr.map(lambda line: line.split(","))

In [16]:
airlinesParsed.take(10)

[['"19031"', '"Mackey International Inc.: MAC"'],
 ['"19032"', '"Munz Northern Airlines Inc.: XY"'],
 ['"19033"', '"Cochise Airlines Inc.: COC"'],
 ['"19034"', '"Golden Gate Airlines Inc.: GSA"'],
 ['"19035"', '"Aeromech Inc.: RZZ"'],
 ['"19036"', '"Golden West Airlines Co.: GLW"'],
 ['"19037"', '"Puerto Rico Intl Airlines: PRN"'],
 ['"19038"', '"Air America Inc.: STZ"'],
 ['"19039"', '"Swift Aire Lines Inc.: SWT"'],
 ['"19040"', '"American Central Airlines: TSF"']]

In [33]:
import csv
from io import StringIO


def split(line):
    reader = csv.reader(StringIO(line))
   
    return next(reader)


airlinesWoHdr.map(split).take(10)

[['19031', 'Mackey International Inc.: MAC'],
 ['19032', 'Munz Northern Airlines Inc.: XY'],
 ['19033', 'Cochise Airlines Inc.: COC'],
 ['19034', 'Golden Gate Airlines Inc.: GSA'],
 ['19035', 'Aeromech Inc.: RZZ'],
 ['19036', 'Golden West Airlines Co.: GLW'],
 ['19037', 'Puerto Rico Intl Airlines: PRN'],
 ['19038', 'Air America Inc.: STZ'],
 ['19039', 'Swift Aire Lines Inc.: SWT'],
 ['19040', 'American Central Airlines: TSF']]

In [82]:
from datetime import datetime
from collections import namedtuple


fields = ('date','airline','flightnum', 'origin','dest','dep','dep_delay','arv'
         ,'arv_delay','airtime','distance')

Flight = namedtuple('Flight',fields,verbose=True)

DATE_FMT = "%Y-%m-%d"


TIME_FMT = "%H%M"

def parse(row):
   
    row[0]=datetime.strptime(row[0],DATE_FMT).date()
    row[5]=datetime.strptime(row[5],TIME_FMT).time()
    row[7]=datetime.strptime(row[7],TIME_FMT).time()
    row[6]=float(row[6])
    row[8]=float(row[8])
    row[9]=float(row[9])
    row[10]=float(row[10])
    return Flight(*row[:11])




from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Flight(tuple):
    'Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'

    __slots__ = ()

    _fields = ('date', 'airline', 'flightnum', 'origin', 'dest', 'dep', 'dep_delay', 'arv', 'arv_delay', 'airtime', 'distance')

    def __new__(_cls, date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance):
        'Create new instance of Flight(date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance)'
        return _tuple.__new__(_cls, (date, airline, flightnum, origin, dest, dep, dep_delay, arv, arv_delay, airtime, distance))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Flight object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 11:
  

In [102]:
flights_rdd = sc.textFile(flightPath)


In [96]:
flightsParsed = flights_rdd.map(lambda item: item.split(",")).map(parse)
flightsParsed.persist()

PythonRDD[57] at RDD at PythonRDD.scala:49

In [85]:
 flightsParsed.first()


Flight(date=datetime.date(2014, 4, 1), airline='19805', flightnum='1', origin='JFK', dest='LAX', dep=datetime.time(8, 54), dep_delay=-6.0, arv=datetime.time(12, 17), arv_delay=2.0, airtime=355.0, distance=2475.0)

In [90]:
total_flights=flightsParsed.count()

In [88]:
totalDistance =flightsParsed.map(lambda obj : obj.distance).reduce(lambda x,y: x+y)

In [91]:
#average distance travelled by all flights
totalDistance/total_flights

794.8585013871385

In [93]:
flightWithDepDelay =flightsParsed.filter(lambda obj: obj.dep_delay>0).count()

In [97]:
#percentage of filghts with departure delay
flightWithDepDelay/total_flights


0.3753871510922012

In [98]:
flightsParsed.unpersist()

PythonRDD[57] at RDD at PythonRDD.scala:49

In [99]:
#compute average delay
sumCount = flightsParsed.map(lambda obj: obj.dep_delay).aggregate((0,0),
                                                                  (lambda acc, value: (acc[0]+ value, acc[1]+1)),
                                                                  (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1] )))

In [100]:
sumCount[0]/sumCount[1]

8.313877046894298

In [103]:
#delay frequency histogram

flightsParsed.map(lambda obj: int(obj.dep_delay/60)).countByValue()

defaultdict(int,
            {0: 452963,
             1: 16016,
             2: 4893,
             3: 1729,
             4: 701,
             5: 249,
             6: 113,
             7: 66,
             8: 43,
             9: 26,
             10: 15,
             11: 12,
             12: 9,
             13: 15,
             14: 13,
             15: 4,
             17: 2,
             20: 4,
             21: 3,
             24: 3,
             25: 1,
             28: 1})

In [105]:
#average delay per airport

airportDelays = flightsParsed.map(lambda obj: (obj.origin, obj.dep_delay))

In [113]:
airportTotalDelays=airportDelays.reduceByKey(lambda x,y:x+y) 

In [114]:
airportTotalDelays.take(10)

[('LAX', 149470.0),
 ('BOS', 49478.0),
 ('SFO', 124901.0),
 ('ATL', 261924.0),
 ('DCA', 27099.0),
 ('ORD', 274709.0),
 ('PBI', 26107.0),
 ('FLL', 63583.0),
 ('IAD', 54591.0),
 ('SJU', 12262.0)]

In [122]:
airportCount = airportDelays.mapValues(lambda x:1).reduceByKey(lambda x,y:x+y) 

In [123]:
airportCount.take(10)

[('LAX', 18359),
 ('BOS', 9292),
 ('SFO', 13529),
 ('ATL', 30919),
 ('DCA', 5834),
 ('ORD', 22724),
 ('PBI', 2176),
 ('FLL', 5801),
 ('IAD', 4875),
 ('SJU', 2184)]

In [126]:
airportSumCount= airportTotalDelays.join(airportCount)
airportSumCount.first()

('LAX', (149470.0, 18359))

In [127]:
airportAvgDelay= airportSumCount.mapValues(lambda x: x[0]/x[1])

In [128]:
airportAvgDelay.take(5)

[('LAX', 8.14151097554333),
 ('BOS', 5.324795523030564),
 ('SFO', 9.23209402025279),
 ('ATL', 8.471295966881206),
 ('DCA', 4.645011998628728)]

In [130]:
#average delay per airport combinebyKey

airportSumCount2= airportDelays.combineByKey((lambda x: (x,1)),
                                            (lambda acc, value: (acc[0]+value,acc[1]+1)),
                                            (lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1] )))


In [131]:
airportSumCount2.take(5)

[('LAX', (149470.0, 18359)),
 ('BOS', (49478.0, 9292)),
 ('SFO', (124901.0, 13529)),
 ('ATL', (261924.0, 30919)),
 ('DCA', (27099.0, 5834))]

In [132]:
airportAvgDelay2= airportSumCount2.mapValues(lambda x: x[0]/x[1])

In [133]:
airportAvgDelay2.take(5)

[('LAX', 8.14151097554333),
 ('BOS', 5.324795523030564),
 ('SFO', 9.23209402025279),
 ('ATL', 8.471295966881206),
 ('DCA', 4.645011998628728)]

In [135]:
#top 10 ariports with worst delay
airportAvgDelay2.sortBy(lambda x: -x[1]).take(10)

[('PPG', 56.25),
 ('EGE', 32.0),
 ('OTH', 24.533333333333335),
 ('LAR', 18.892857142857142),
 ('RDD', 18.55294117647059),
 ('MTJ', 18.363636363636363),
 ('PUB', 17.54),
 ('EWR', 16.478549005929544),
 ('CIC', 15.931034482758621),
 ('RST', 15.6993006993007)]

In [146]:
airports = sc.textFile(airportsPath)

In [137]:
airports.take(2)

['Code,Description', '"01A","Afognak Lake, AK: Afognak Lake Airport"']

In [151]:
airports =airports.filter(lambda line : "Description" not in line).map(split)

In [154]:
airportsLookup=airports.collectAsMap()

In [153]:
airports.lookup('PPG')

['Pago Pago, TT: Pago Pago International']

In [155]:
airportsLookup

{'01A': 'Afognak Lake, AK: Afognak Lake Airport',
 '03A': 'Granite Mountain, AK: Bear Creek Mining Strip',
 '04A': 'Lik, AK: Lik Mining Camp',
 '05A': 'Little Squaw, AK: Little Squaw Airport',
 '06A': 'Kizhuyak, AK: Kizhuyak Bay',
 '07A': 'Klawock, AK: Klawock Seaplane Base',
 '08A': 'Elizabeth Island, AK: Elizabeth Island Airport',
 '09A': 'Homer, AK: Augustin Island',
 '1B1': 'Hudson, NY: Columbia County',
 '1G4': 'Peach Springs, AZ: Grand Canyon West',
 '1N7': 'Blairstown, NJ: Blairstown Airport',
 '1NY': 'Penn Yan, NY: Penn Yan Airport',
 '6B0': 'Middlebury, VT: Middlebury State',
 '7AK': 'Akun, AK: Akun Airport',
 '8F3': 'Crosbyton, TX: Crosbyton Municipal',
 'A01': 'Fairbanks/Ft. Wainwright, AK: Blair Lake',
 'A02': 'Deadmans Bay, AK: Deadmans Bay Airport',
 'A03': 'Hallo Bay, AK: Hallo Bay Airport',
 'A04': 'Red Lake, AK: Red Lake Airport',
 'A05': 'Shell Lake, AK: Shell Lake Airport',
 'A06': 'Kougarok, AK: Navigator Airstrip',
 'A07': 'Selawik, AK: Roland Norton Memorial',
 'A

In [158]:
airportAvgDelayWithName= airportAvgDelay2.map(lambda x: (airportsLookup[x[0]],x[1]))

In [159]:
airportAvgDelayWithName.take(2)

[('Los Angeles, CA: Los Angeles International', 8.14151097554333),
 ('Boston, MA: Logan International', 5.324795523030564)]

In [163]:
#broadcast variable

ariportsBC = sc.broadcast(airportsLookup)

In [161]:
airportAvgDelayWithName= airportAvgDelay2.map(lambda x: (ariportsBC.value([x[0]]),x[1]))

In [162]:
airportAvgDelayWithName.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 101.0 failed 1 times, most recent failure: Lost task 0.0 in stage 101.0 (TID 111, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/madsh01/workspace/dist/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/Users/madsh01/workspace/dist/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/madsh01/workspace/dist/spark-2.3.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/madsh01/workspace/dist/spark-2.3.1-bin-hadoop2.7/python/pyspark/rdd.py", line 1371, in takeUpToN